In [1]:
import numpy as np

import sys
sys.path.append("../")
from koopmanlib.dictionary import PsiNN

In [2]:
basis_function = PsiNN(layer_sizes=[100,100,100], n_psi_train=100)

In [3]:
from ou_process import ou_process_1d

theta = 1
mu = 0
sigma = 1
step_num = 10000
h = 1e-5
# X0 = np.random.uniform(-2, 2) # initial point


N = 10000  # Number of the initial points
state = np.random.RandomState()
initial_conditions = state.uniform(-2, 2, size=N)
data_initial = initial_conditions

### Parallel computation ###

# Here we use 8 cores to speed up the computation 
from joblib import Parallel, delayed

def compute_trajectory_1d(initial_conditions):
    X0 = initial_conditions
    X_traj = ou_process_1d(theta, mu, sigma, X0, step_num, h)
    return X_traj[-1]  # Return both initial and final states

# Use 8 cores
n_jobs = 8
results = Parallel(n_jobs=n_jobs)(delayed(compute_trajectory_1d)(ic) for ic in initial_conditions)

# Convert results (which is a list of tuples) into desired format
data_final = np.array(results)  # Shape: (N, 1)

# Pairwise shifted data X and Y
X = data_initial.reshape(N,1) # shape: (10000, 1)
Y = data_final.reshape(N,1)

In [4]:
len_all = X.shape[0]
data_x_train = X[:int(0.7*len_all),:]
data_x_valid = X[int(0.7*len_all)+1:,:]

data_y_train = Y[:int(0.7*len_all),:]
data_y_valid = Y[int(0.7*len_all)+1:,:]

In [5]:
data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

In [14]:
from koopmanlib.solver_0 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-6,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 1s 165ms/step - loss: 0.0102 - val_loss: 0.0106
Epoch 2/2
2/2 [==============================] - 0s 39ms/step - loss: 0.0102 - val_loss: 0.0106
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 62ms/step - loss: 0.0102 - val_loss: 0.0106
Epoch 2/2
2/2 [==============================] - 0s 39ms/step - loss: 0.0102 - val_loss: 0.0106
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 65ms/step - loss: 0.0102 - val_loss: 0.0105
Epoch 2/2
2/2 [==============================] - 0s 38ms/step - loss: 0.0102 - val_loss: 0.0105
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 65ms/step - loss: 0.0102 - val_loss: 0.0105
Epoch 2/2
2/2 [==============================] - 0s 40ms/step - loss: 0.0102 - val_loss: 0.0105
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 62ms/step - loss: 0.0102 - val_loss: 0.0105
Epoch 2/2
2/2 [========

In [15]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs0 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs0['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs0['resDMD_DL_outputs']['efuns'].shape)

[ 1.00000216e+00+0.00000000e+00j  9.08889834e-01+0.00000000e+00j
  7.29246960e-01+0.00000000e+00j  6.10766145e-01+0.00000000e+00j
  3.94597471e-01+0.00000000e+00j  2.48982390e-03+0.00000000e+00j
  8.11875149e-04+0.00000000e+00j  3.74212917e-06+0.00000000e+00j
  1.28861485e-06+0.00000000e+00j  2.78811791e-09+0.00000000e+00j
  9.79544017e-10+0.00000000e+00j  1.29184679e-10+0.00000000e+00j
  5.29494123e-11+0.00000000e+00j  2.56206026e-11+7.03407614e-12j
  2.56206026e-11-7.03407614e-12j  2.37953260e-11+0.00000000e+00j
  1.84366336e-11-1.90234202e-11j  1.84366336e-11+1.90234202e-11j
  1.12832559e-11+1.02794680e-11j  1.12832559e-11-1.02794680e-11j
  1.00501623e-11+2.02559410e-12j  1.00501623e-11-2.02559410e-12j
  9.70307982e-12-3.18501552e-11j  9.70307982e-12+3.18501552e-11j
  7.82912347e-12+0.00000000e+00j  6.26347818e-12+0.00000000e+00j
  6.11534489e-12-3.90279273e-12j  6.11534489e-12+3.90279273e-12j
  6.06670211e-12+1.42699533e-11j  6.06670211e-12-1.42699533e-11j
  3.99971654e-12+0.000000

In [8]:
from koopmanlib.solver_1 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-6,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 27s 4s/step - loss: 0.7817 - val_loss: 0.7981
Epoch 2/2
2/2 [==============================] - 1s 285ms/step - loss: 0.7862 - val_loss: 0.8000
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 1s 322ms/step - loss: 0.7834 - val_loss: 0.7935
Epoch 2/2
2/2 [==============================] - 0s 294ms/step - loss: 0.7822 - val_loss: 0.7955
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 308ms/step - loss: 0.7849 - val_loss: 0.7962
Epoch 2/2
2/2 [==============================] - 0s 291ms/step - loss: 0.7854 - val_loss: 0.8051
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 309ms/step - loss: 0.7820 - val_loss: 0.7975
Epoch 2/2
2/2 [==============================] - 0s 281ms/step - loss: 0.7844 - val_loss: 0.8021
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 1s 322ms/step - loss: 0.7840 - val_loss: 0.7971
Epoch 2/2
2/2 [==

In [9]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs1 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs1['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs1['resDMD_DL_outputs']['efuns'].shape)

[ 9.99998599e-01+0.00000000e+00j  9.08351443e-01+0.00000000e+00j
  7.26430781e-01+0.00000000e+00j  5.42528073e-01+0.00000000e+00j
  4.74203550e-02+0.00000000e+00j  1.52504864e-02+0.00000000e+00j
  7.07569618e-05+0.00000000e+00j  4.41768582e-05+0.00000000e+00j
  3.83178902e-07+0.00000000e+00j  7.30638376e-08+0.00000000e+00j
  7.93595837e-10+0.00000000e+00j  3.71604133e-10+1.83937227e-10j
  3.71604133e-10-1.83937227e-10j  3.46270641e-10+0.00000000e+00j
  2.34630947e-10+0.00000000e+00j  2.01233884e-10+0.00000000e+00j
  1.91744754e-10+1.77597242e-10j  1.91744754e-10-1.77597242e-10j
  1.66598978e-10+0.00000000e+00j  1.41768226e-10-9.40158522e-11j
  1.41768226e-10+9.40158522e-11j  1.13346877e-10-3.46127040e-11j
  1.13346877e-10+3.46127040e-11j  1.09088630e-10-7.33744183e-11j
  1.09088630e-10+7.33744183e-11j  1.05928736e-10+2.14269121e-10j
  1.05928736e-10-2.14269121e-10j  1.03646311e-10+0.00000000e+00j
  9.37789300e-11+1.46405850e-10j  9.37789300e-11-1.46405850e-10j
  7.43789650e-11-7.879374

In [12]:
from koopmanlib.solver_2 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-6,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 1s 262ms/step - loss: 3598215.8214 - val_loss: 12311554.0000
Epoch 2/2
2/2 [==============================] - 0s 108ms/step - loss: 6535216.6429 - val_loss: 12316893.0000
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 139ms/step - loss: 25678524.1429 - val_loss: 12307334.0000
Epoch 2/2
2/2 [==============================] - 0s 106ms/step - loss: 4201168.5000 - val_loss: 12297506.0000
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 152ms/step - loss: 3218251.8929 - val_loss: 12288833.0000
Epoch 2/2
2/2 [==============================] - 0s 110ms/step - loss: 9218055.2857 - val_loss: 12277001.0000
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 138ms/step - loss: 3991237.8214 - val_loss: 12266502.0000
Epoch 2/2
2/2 [==============================] - 0s 110ms/step - loss: 4012201.5357 - val_loss: 12257712.0000
number of the outer loop: 3
Epoch 1

In [13]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs2 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs2['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs2['resDMD_DL_outputs']['efuns'].shape)

[ 9.99998666e-01+0.00000000e+00j  9.08352744e-01+0.00000000e+00j
  7.26460459e-01+0.00000000e+00j  5.42309871e-01+0.00000000e+00j
  5.67830863e-02+0.00000000e+00j  4.44283692e-02+0.00000000e+00j
  2.17473257e-04+0.00000000e+00j  5.58516814e-05+0.00000000e+00j
  4.12298244e-07+0.00000000e+00j  2.29148340e-07+0.00000000e+00j
  8.67837452e-10+0.00000000e+00j  4.32328406e-10+0.00000000e+00j
  3.09727439e-10-9.84013787e-11j  3.09727439e-10+9.84013787e-11j
  2.77737041e-10-1.97924539e-10j  2.77737041e-10+1.97924539e-10j
  2.76579159e-10+0.00000000e+00j  1.89207098e-10-2.45589425e-10j
  1.89207098e-10+2.45589425e-10j  1.70724796e-10+1.51986940e-11j
  1.70724796e-10-1.51986940e-11j  1.55473658e-10-1.32502150e-10j
  1.55473658e-10+1.32502150e-10j  1.44437834e-10-6.52546881e-11j
  1.44437834e-10+6.52546881e-11j  1.08276741e-10+0.00000000e+00j
  7.15944240e-11+3.14403874e-11j  7.15944240e-11-3.14403874e-11j
  5.64009928e-11+0.00000000e+00j  3.97095474e-11+0.00000000e+00j
  3.55457216e-11+5.842673

In [6]:
from koopmanlib.solver_3 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-6,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 1s 268ms/step - loss: 0.0105 - val_loss: 0.0121
Epoch 2/2
2/2 [==============================] - 0s 100ms/step - loss: 0.0049 - val_loss: 0.0121
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 123ms/step - loss: 0.0043 - val_loss: 0.0121
Epoch 2/2
2/2 [==============================] - 0s 100ms/step - loss: 0.0041 - val_loss: 0.0121
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 120ms/step - loss: 0.0124 - val_loss: 0.0121
Epoch 2/2
2/2 [==============================] - 0s 108ms/step - loss: 0.0049 - val_loss: 0.0121
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 120ms/step - loss: 0.0077 - val_loss: 0.0121
Epoch 2/2
2/2 [==============================] - 0s 98ms/step - loss: 0.0053 - val_loss: 0.0121
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 122ms/step - loss: 0.0061 - val_loss: 0.0121
Epoch 2/2
2/2 [=

In [7]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs3 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs3['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs3['resDMD_DL_outputs']['efuns'].shape)

[ 9.99998598e-01+0.00000000e+00j  9.08351242e-01+0.00000000e+00j
  7.26376205e-01+0.00000000e+00j  5.42717721e-01+0.00000000e+00j
  4.74255756e-02+0.00000000e+00j  1.55537134e-02+0.00000000e+00j
  5.65422292e-05+2.36364373e-06j  5.65422292e-05-2.36364373e-06j
  3.66561255e-07+0.00000000e+00j  8.13822805e-08+0.00000000e+00j
  9.28941245e-10+0.00000000e+00j  2.77382652e-10+8.78703365e-11j
  2.77382652e-10-8.78703365e-11j  2.63860149e-10+2.06963130e-10j
  2.63860149e-10-2.06963130e-10j  1.89657971e-10+7.94166302e-11j
  1.89657971e-10-7.94166302e-11j  1.63237494e-10+1.91624979e-10j
  1.63237494e-10-1.91624979e-10j  1.42077501e-10+1.06351686e-10j
  1.42077501e-10-1.06351686e-10j  1.30257216e-10+3.99055330e-11j
  1.30257216e-10-3.99055330e-11j  1.11267463e-10+0.00000000e+00j
  9.40932071e-11+0.00000000e+00j  6.85062515e-11-9.85137105e-11j
  6.85062515e-11+9.85137105e-11j  6.80678194e-11+2.75816599e-11j
  6.80678194e-11-2.75816599e-11j  6.04196981e-11+3.91612449e-11j
  6.04196981e-11-3.916124

In [10]:
from koopmanlib.solver_4 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-6,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 1s 295ms/step - loss: 49307.4074 - val_loss: 38847.1836
Epoch 2/2
2/2 [==============================] - 0s 117ms/step - loss: 21726.5241 - val_loss: 55272.5039
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 149ms/step - loss: 32972.0907 - val_loss: 40075.5078
Epoch 2/2
2/2 [==============================] - 0s 114ms/step - loss: 81796.1775 - val_loss: 33313430.0000
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 140ms/step - loss: 16794.3790 - val_loss: 176902.3906
Epoch 2/2
2/2 [==============================] - 0s 128ms/step - loss: 222251.2210 - val_loss: 29008.5527
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 140ms/step - loss: 27764.0167 - val_loss: 26715.1055
Epoch 2/2
2/2 [==============================] - 0s 122ms/step - loss: 31083.9805 - val_loss: 32847.6562
number of the outer loop: 3
Epoch 1/2
2/2 [============================

In [11]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs4 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs4['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs4['resDMD_DL_outputs']['efuns'].shape)

[ 9.99998604e-01+0.00000000e+00j  9.08351521e-01+0.00000000e+00j
  7.26413053e-01+0.00000000e+00j  5.42549036e-01+0.00000000e+00j
  4.66185714e-02+0.00000000e+00j  1.97367091e-02+0.00000000e+00j
  6.85544852e-05+0.00000000e+00j  5.78441004e-05+0.00000000e+00j
  3.52157322e-07+0.00000000e+00j  7.56939921e-08+0.00000000e+00j
  7.93983996e-10+0.00000000e+00j  4.26614462e-10+6.25959755e-11j
  4.26614462e-10-6.25959755e-11j  2.87546284e-10+0.00000000e+00j
  2.25658973e-10-1.06676632e-10j  2.25658973e-10+1.06676632e-10j
  2.19117222e-10-2.84603074e-10j  2.19117222e-10+2.84603074e-10j
  1.54372618e-10-1.83047824e-10j  1.54372618e-10+1.83047824e-10j
  1.48704883e-10+0.00000000e+00j  1.44099144e-10+9.67918594e-11j
  1.44099144e-10-9.67918594e-11j  1.31863309e-10-2.91428432e-10j
  1.31863309e-10+2.91428432e-10j  1.07205764e-10+7.53751410e-11j
  1.07205764e-10-7.53751410e-11j  7.45074272e-11+4.77192324e-11j
  7.45074272e-11-4.77192324e-11j  7.13946834e-11-1.39237985e-10j
  7.13946834e-11+1.392379